In [1]:
using Pkg
Pkg.activate(pwd())

  Activating project at `c:\Users\pecci\Code\MACRO`


In [2]:
using JuMP,HiGHS

In [3]:
abstract type Network end

In [4]:
abstract type Process end

In [5]:
abstract type AbstractNode end

In [7]:
Base.@kwdef mutable struct Transformation <: Process
    CommodityIn::String
    CommodityOut::String
    NodeIn::Int64
    NodeOut::Int64
    MinCap::Float64 = 0.0
    MaxCap::Float64 = 100.0
    CanExpand::Bool = true
    CanRetire::Bool = true
    ExistingCap::Float64 = 0.0
    RampingUpRate::Float64 = 1.0
    RampingDnRate::Float64 = 1.0
    MinPower::Float64 = 0.0
    ConversionRate::Dict{String,Float64}
    MinUpTime::Float64 = 0.0
    MinDnTime::Float64 = 0.0
    JumpInvVars::Dict = Dict()
    JumpOpVars::Dict = Dict()
    InvCost::Float64 = 0.0
    FOMCost::Float64 = 0.0
    VOMCost::Float64 = 0.0
end

Transformation

In [8]:
Base.@kwdef mutable struct Storage <: Process
    Commodity::String
    Node::Int64
    EffUp::Float64 = 1.0
    EffDn::Float64 = 1.0
    SelfDischarge::Float64 = 0.0
    MinStorageLevel::Float64 = 0.0
    MinCap::Float64 = 0.0
    MaxCap::Float64 = 100.0
    CanExpand::Bool = true
    CanRetire::Bool = true
    JumpInvVars::Dict = Dict()
    JumpOpVars::Dict = Dict()
    InvCost_discharge::Float64 = 0.0
    InvCost_charge::Float64 = 0.0
    InvCost_storage::Float64 = 0.0
    FOMCost_discharge::Float64 = 0.0
    FOMCost_charge::Float64 = 0.0
    FOMCost_storage::Float64 = 0.0
    VOMCost_discharge::Float64 = 0.0 
    VOMCost_charge::Float64 = 0.0
end

Storage

In [9]:
Base.@kwdef mutable struct Resource <: Process
    Commodity::String
    Node::Int64
    MinCap::Float64 = 0.0
    MaxCap::Float64 = 100.0
    ExistingCap::Float64 = 0.0
    PolicyMembership::Vector{String} = [""]
    CanExpand::Bool = true
    CanRetire::Bool = true
    CapacityFactor::Vector{Float64}
    JumpInvVars::Dict = Dict()
    JumpOpVars::Dict = Dict()
    InvCost::Float64 = 0.0;
    FOMCost::Float64 = 0.0;
    VOMCost::Float64 = 0.0;
end

Resource

In [10]:
Base.@kwdef mutable struct Node <: AbstractNode
    Index::Int64
    Commodity::String
    TimeSeries::Vector{Float64}
end    

Node

In [11]:
Base.@kwdef mutable struct Edge <: Network
    Commodity::String
    Start::Int64
    End::Int64
    Unidirectional::Bool = false
    MinCap::Float64 = 0.0
    MaxCap::Float64 = 100.0
    ExistingCap::Float64 = 100.0
    CanExpand::Bool = true
    CanRetire::Bool = true
    JumpInvVars::Dict = Dict()
    JumpOpVars::Dict = Dict()
    InvCost::Float64 = 0.0;
    FOMCost::Float64 = 0.0;
    VOMCost::Float64 = 0.0;
end

Edge

In [87]:
typeof((1,2))

Tuple{Int64, Int64}

In [12]:
solar_1 = Resource(Commodity = "P",Node = 1,CapacityFactor = rand(10),InvCost=85300.0,FOMCost =18760);
wind_1 = Resource(Commodity = "P",Node = 1,CapacityFactor = rand(10),InvCost=97200.0,FOMCost=43205.0,VOMCost=0.1);
battery_1 = Storage(Commodity="P",Node = 1,EffUp = 0.8,EffDn = 0.8,SelfDischarge = 0.02,InvCost_discharge=19584.0,FOMCost_discharge=4895.0,VOMCost_discharge=0.15,InvCost_storage = 22494.0, FOMCost_storage = 5622.0, VOMCost_charge=0.15,);
ElectrolyzerTransform_1 = Transformation(CommodityIn = "P", CommodityOut="H2" ,NodeIn = 1, NodeOut = 1 ,ConversionRate = Dict("P"=>1.0,"H2"=>-0.6),InvCost=125000.0,FOMCost=15000.0);

solar_2 = Resource(Commodity = "P",Node = 2,CapacityFactor = rand(10),InvCost=85300.0,FOMCost =18760);
wind_2 = Resource(Commodity = "P",Node = 2,CapacityFactor = rand(10),InvCost=97200.0,FOMCost=43205.0,VOMCost=0.1);
battery_2 = Storage(Commodity="P",Node = 2,EffUp = 0.8,EffDn = 0.8,SelfDischarge = 0.02,InvCost_discharge=19584.0,FOMCost_discharge=4895.0,VOMCost_discharge=0.15,InvCost_storage = 22494.0, FOMCost_storage = 5622.0, VOMCost_charge=0.15,);
ElectrolyzerTransform_2 = Transformation(CommodityIn = "P", CommodityOut="H2" ,NodeIn = 2, NodeOut = 2 ,ConversionRate = Dict("P"=>1.0,"H2"=>-0.6),InvCost=125000.0,FOMCost=15000.0);

solar_3 = Resource(Commodity = "P",Node = 3,CapacityFactor = rand(10),InvCost=85300.0,FOMCost =18760);
wind_3 = Resource(Commodity = "P",Node = 3,CapacityFactor = rand(10),InvCost=97200.0,FOMCost=43205.0,VOMCost=0.1);
battery_3 = Storage(Commodity="P",Node = 3,EffUp = 0.8,EffDn = 0.8,SelfDischarge = 0.02,InvCost_discharge=19584.0,FOMCost_discharge=4895.0,VOMCost_discharge=0.15,InvCost_storage = 22494.0, FOMCost_storage = 5622.0, VOMCost_charge=0.15,);
ElectrolyzerTransform_3 = Transformation(CommodityIn = "P", CommodityOut="H2" ,NodeIn = 3, NodeOut = 3 ,ConversionRate = Dict("P"=>1.0,"H2"=>-0.6),InvCost=125000.0,FOMCost=15000.0);

network = Dict("P"=>[Edge(Start=1,End=3,Commodity="P");Edge(Start=2,End=3,Commodity="P")],"H2"=>[Edge(Start=1,End=2,Unidirectional=true,Commodity="H2");Edge(Start=2,End=3,Unidirectional=true,Commodity="H2")]);

In [13]:
technology_map = Dict("P"=>Dict(1=>[solar_1,wind_1,battery_1],2=>[solar_2,wind_2,battery_2],3=>[solar_3,wind_3,battery_3]),"H2"=>Dict(1=>[],2=>[],3=>[]))
time_domain_map = Dict("P"=>collect(1:10),"H2"=>collect(1:10))
demand_map = Dict("P"=>Dict(1=>Node(1,"P",7.5*rand(10)),2=>Node(2,"P",3.5*rand(10)),3=>Node(3,"P",0*rand(10))),"H2"=>Dict(1=>Node(1,"H2",7.5*rand(10)),2=>Node(2,"H2",0*rand(10)),3=>Node(3,"H2",3.5*rand(10))));
VOLL = Dict("P"=>5000,"H2"=>5000)

Dict{String, Int64} with 2 entries:
  "P"  => 5000
  "H2" => 5000

In [14]:
technology_map["P"][1]

3-element Vector{Process}:
 Resource("P", 1, 0.0, 100.0, 0.0, [""], true, true, [0.7436877762167187, 0.422797389746686, 0.15520062303235016, 0.3949407533995368, 0.34709398310009565, 0.7896972925404478, 0.9390336452450303, 0.2048437366643151, 0.9454265037934545, 0.026070644136222287], Dict{Any, Any}(), Dict{Any, Any}(), 85300.0, 18760.0, 0.0)
 Resource("P", 1, 0.0, 100.0, 0.0, [""], true, true, [0.42967054536701643, 0.24673830354186765, 0.17387999933529064, 0.7672388930571827, 0.41714259920257446, 0.469114671518315, 0.37334780194043515, 0.036860398300826325, 0.19718777069162374, 0.8695394483597289], Dict{Any, Any}(), Dict{Any, Any}(), 97200.0, 43205.0, 0.1)
 Storage("P", 1, 0.8, 0.8, 0.02, 0.0, 0.0, 100.0, true, true, Dict{Any, Any}(), Dict{Any, Any}(), 19584.0, 0.0, 22494.0, 4895.0, 0.0, 5622.0, 0.15, 0.15)

In [15]:
function add_resource!(EP::JuMP.Model,g::Resource,num_inv_vars::Int64,num_op_vars::Int64,T::Vector{Int64})

    g.JumpInvVars["GenCap"] = JuMP.@variable(EP,lower_bound=0,base_name="y[$(num_inv_vars+1)]"); 
  
    JuMP.@constraint(EP,g.JumpInvVars["GenCap"] <=g.MaxCap)

    JuMP.@constraint(EP,g.JumpInvVars["GenCap"] >=g.MinCap)
    
    g.JumpOpVars["Gen"] = Dict(t=>JuMP.@variable(EP,lower_bound=0,base_name="x[$(num_op_vars+1),$t]") for t in T); 
    
    JuMP.@constraint(EP,[t in T], g.JumpOpVars["Gen"][t]<=g.CapacityFactor[t]*g.JumpInvVars["GenCap"] );
    
    #### Energy balance contribution
    for t in T
        EP[:eEnergyBalance][g.Commodity,g.Node,t] += g.JumpOpVars["Gen"][t];
    end
   
    EP[:eTotalCost] += g.InvCost*g.JumpInvVars["GenCap"] + g.FOMCost*g.JumpInvVars["GenCap"] + sum(g.VOMCost*g.JumpOpVars["Gen"][t] for t in T);
    
end

add_resource! (generic function with 1 method)

In [16]:
function add_storage!(EP::JuMP.Model,g::Storage,num_inv_vars::Int64,num_op_vars::Int64,T::Vector{Int64})

    g.JumpInvVars["GenCap"] = JuMP.@variable(EP,lower_bound=0,base_name="y[$(num_inv_vars+1)]"); #charge/discharge capacity
    g.JumpInvVars["StorCap"]  = JuMP.@variable(EP,lower_bound=0,base_name="y[$(num_inv_vars+2)]"); #state of charge capacity
    
    g.JumpOpVars["Gen"] = Dict(t=>JuMP.@variable(EP,lower_bound=0,base_name="x[$(num_op_vars+1),$t]") for t in T); #discharge
    g.JumpOpVars["Charge"] = Dict(t=>JuMP.@variable(EP,lower_bound=0,base_name="x[$(num_op_vars+2),$t]") for t in T); #charge
    g.JumpOpVars["StorLevel"] = Dict(t=>JuMP.@variable(EP,lower_bound=0,base_name="x[$(num_op_vars+3),$t]") for t in T); #state of charge

    JuMP.@constraint(EP,g.JumpInvVars["GenCap"]<=g.MaxCap)

    JuMP.@constraint(EP,g.JumpInvVars["GenCap"]>=g.MinCap)

    JuMP.@constraint(EP,[t in T], g.JumpOpVars["Gen"][t]<=g.JumpInvVars["GenCap"]);

    JuMP.@constraint(EP,[t in T], g.JumpOpVars["Charge"][t]<=g.JumpInvVars["GenCap"]);

    JuMP.@constraint(EP,[t in T], g.JumpOpVars["StorLevel"][t]<=g.JumpInvVars["StorCap"]);

   
    for t in T
        
        if t==1
            JuMP.@constraint(EP,g.JumpOpVars["StorLevel"][t]== (1-g.SelfDischarge)*g.JumpOpVars["StorLevel"][T[end]] + g.EffUp*g.JumpOpVars["Charge"][t] - 1/g.EffDn*g.JumpOpVars["Gen"][t]);
        else
            JuMP.@constraint(EP,g.JumpOpVars["StorLevel"][t]== (1-g.SelfDischarge)*g.JumpOpVars["StorLevel"][t-1] + g.EffUp*g.JumpOpVars["Charge"][t] - 1/g.EffDn*g.JumpOpVars["Gen"][t]);
        end
        
        EP[:eEnergyBalance][g.Commodity,g.Node,t] += g.JumpOpVars["Gen"][t] - g.JumpOpVars["Charge"][t];
        
    end

    JuMP.@constraint(EP,[t in T], g.JumpOpVars["StorLevel"][t]>=g.MinStorageLevel);
   
    EP[:eTotalCost] += g.InvCost_discharge*g.JumpInvVars["GenCap"] + g.FOMCost_discharge*g.JumpInvVars["GenCap"] + g.InvCost_storage*g.JumpInvVars["StorCap"] + g.FOMCost_storage*g.JumpInvVars["StorCap"] + sum(g.VOMCost_charge*g.JumpOpVars["Charge"][t] for t in T) +  sum(g.VOMCost_discharge*g.JumpOpVars["Gen"][t] for t in T);
    
end

add_storage! (generic function with 1 method)

In [21]:
function add_transformation!(EP::JuMP.Model,g::Transformation,num_inv_vars::Int64,num_op_vars::Int64,T::Vector{Int64})

    g.JumpInvVars["InputCap"] = JuMP.@variable(EP,lower_bound=0,base_name="y[$(num_inv_vars+1)]"); 
    g.JumpInvVars["OutputCap"]  = JuMP.@variable(EP,lower_bound=0,base_name="y[$(num_inv_vars+2)]"); 
    
    g.JumpOpVars["Input"] = Dict(t=>JuMP.@variable(EP,lower_bound=0,base_name="x[$(num_op_vars+1),$t]") for t in T); 
    g.JumpOpVars["Output"] = Dict(t=>JuMP.@variable(EP,lower_bound=0,base_name="x[$(num_op_vars+2),$t]") for t in T);

    JuMP.@constraint(EP, g.JumpInvVars["InputCap"]>=g.MinCap)
    JuMP.@constraint(EP, g.JumpInvVars["OutputCap"]>=g.MinCap)
    JuMP.@constraint(EP, g.JumpInvVars["InputCap"]<=g.MaxCap)
    JuMP.@constraint(EP, g.JumpInvVars["OutputCap"]<=g.MaxCap)

    JuMP.@constraint(EP,[t in T],g.JumpOpVars["Input"][t]<=g.JumpInvVars["InputCap"])
    JuMP.@constraint(EP,[t in T],g.JumpOpVars["Output"][t]<=g.JumpInvVars["OutputCap"])

    JuMP.@constraint(EP,[t in T],g.ConversionRate[g.CommodityIn]*g.JumpOpVars["Input"][t] + g.ConversionRate[g.CommodityOut]*g.JumpOpVars["Output"][t] == 0)

    for t in T

        EP[:eEnergyBalance][g.CommodityIn,g.NodeIn,t] -= g.JumpOpVars["Input"][t];
        
        EP[:eEnergyBalance][g.CommodityOut,g.NodeOut,t] += g.JumpOpVars["Output"][t];

    end

    EP[:eTotalCost] += g.InvCost*(g.JumpInvVars["InputCap"] +  g.JumpInvVars["OutputCap"]) + g.FOMCost*(g.JumpInvVars["InputCap"] + g.JumpInvVars["OutputCap"]) +  sum(g.VOMCost*g.JumpOpVars["Input"][t] for t in T) + sum(g.VOMCost*g.JumpOpVars["Output"][t] for t in T)

end

add_transformation! (generic function with 1 method)

In [18]:
function add_edge!(EP::JuMP.Model,e::Edge,num_inv_vars::Int64,num_op_vars::Int64,T::Vector{Int64})

    e.JumpInvVars["FlowCap"] = JuMP.@variable(EP,lower_bound=0,base_name="y[$(num_inv_vars+1)]"); #flow capacity

    JuMP.@constraint(EP,e.JumpInvVars["FlowCap"]<=e.MaxCap)

    JuMP.@constraint(EP,e.JumpInvVars["FlowCap"]>=e.MinCap)

    if e.Unidirectional
        e.JumpOpVars["Flow"] = Dict(t=>JuMP.@variable(EP,lower_bound=0,base_name="x[$(num_op_vars+1),$t]") for t in T); #flow
        JuMP.@constraint(EP,[t in T], e.JumpOpVars["Flow"][t]<=e.JumpInvVars["FlowCap"]);
    else
        e.JumpOpVars["Flow"] = Dict(t=>JuMP.@variable(EP,base_name="x[$(num_op_vars+1),$t]") for t in T); #flow
        JuMP.@constraint(EP,[t in T], e.JumpOpVars["Flow"][t]<=e.JumpInvVars["FlowCap"]);
        JuMP.@constraint(EP,[t in T], e.JumpOpVars["Flow"][t]>=-e.JumpInvVars["FlowCap"]);
    end

   
    for t in T

        EP[:eEnergyBalance][e.Commodity,e.Start,t] -= e.JumpOpVars["Flow"][t];
        
        EP[:eEnergyBalance][e.Commodity,e.End,t] += e.JumpOpVars["Flow"][t];

    end

    EP[:eTotalCost] += (e.InvCost + e.FOMCost)*e.JumpInvVars["FlowCap"];

end

add_edge! (generic function with 1 method)

In [19]:
network

Dict{String, Vector{Edge}} with 2 entries:
  "P"  => [Edge("P", 1, 3, false, 0.0, 100.0, 100.0, true, true, Dict{Any, Any}…
  "H2" => [Edge("H2", 1, 2, true, 0.0, 100.0, 100.0, true, true, Dict{Any, Any}…

In [97]:
for v in network["P"]
    println(v)
end

Edge("P", (1, 3), false, 0.0, 100.0, 100.0, true, true, Dict{Any, Any}(), Dict{Any, Any}(), 0.0, 0.0, 0.0)
Edge("P", (2, 3), false, 0.0, 100.0, 100.0, true, true, Dict{Any, Any}(), Dict{Any, Any}(), 0.0, 0.0, 0.0)


In [22]:
EP = JuMP.Model(HiGHS.Optimizer);
JuMP.@variable(EP,vZERO==1);
@expression(EP,eEnergyBalance[c in keys(demand_map),n in keys(demand_map[c]),t in time_domain_map[c]],0*EP[:vZERO]);
@expression(EP,eTotalCost,0*EP[:vZERO]);

num_inv_vars = 0;
num_op_vars = 0;

for c in keys(technology_map)
    for n in keys(technology_map[c])
        for g in technology_map[c][n]
            if typeof(g)==Resource
                add_resource!(EP,g,num_inv_vars,num_op_vars,time_domain_map[c])
                num_inv_vars=num_inv_vars+1;
                num_op_vars=num_op_vars+1;
            elseif typeof(g)==Storage
                add_storage!(EP,g,num_inv_vars,num_op_vars,time_domain_map[c])
                num_inv_vars=num_inv_vars+2;
                num_op_vars=num_op_vars+3;
            end
        end
    end
end 
for c in keys(network)
    for v in network[c]
        add_edge!(EP,v,num_inv_vars,num_op_vars,time_domain_map[c])
        num_inv_vars=num_inv_vars+1;
        num_op_vars=num_op_vars+1;
    end
end

add_transformation!(EP,ElectrolyzerTransform_1,num_inv_vars,num_op_vars,time_domain_map["P"])
add_transformation!(EP,ElectrolyzerTransform_2,num_inv_vars,num_op_vars,time_domain_map["P"])
add_transformation!(EP,ElectrolyzerTransform_3,num_inv_vars,num_op_vars,time_domain_map["P"])
num_inv_vars=num_inv_vars+6;
num_op_vars=num_op_vars+6;

JuMP.@variable(EP,vNSE[c in keys(demand_map), n in keys(demand_map[c]),t in time_domain_map[c]]>=0)
JuMP.@constraint(EP,[c in keys(demand_map), n in keys(demand_map[c]),t in time_domain_map[c]], eEnergyBalance[c,n,t] == demand_map[c][n].TimeSeries[t] + vNSE[c,n,t]);
@objective(EP,Min,EP[:eTotalCost]+sum(VOLL[c]*sum(sum(vNSE[c,n,t] for t in time_domain_map[c]) for n in keys(demand_map[c])) for c in keys(demand_map)));

In [23]:
optimize!(EP)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
300 rows, 298 cols, 860 nonzeros
290 rows, 278 cols, 830 nonzeros
Presolve : Reductions: rows 290(-168); columns 278(-55); elements 830(-278)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.7710973026e-04 Pr: 60(3534.52) 0s
        246     4.8338803920e+06 Pr: 0(0); Du: 0(3.05102e-10) 0s
Solving the original LP from the solution after postsolve
JuMP.Model   status      : Optimal
Simplex   iterations: 246
Objective value     :  4.8338803920e+06
HiGHS run time      :          0.02
